In [31]:
import os

import pandas as pd

dump_path = '/home/local/work/data/hgsvc/assmgaps/raw_dumps'


def determine_python_type(sql_type):
    if 'varchar' in sql_type or 'char' in sql_type:
        pytype = str
    elif 'int' in sql_type or 'smallint' in sql_type:
        pytype = int
    elif 'float' in sql_type:
        pytype = float
    else:
        raise ValueError('Cannot determine type: {}'.format(sql_type))
    return pytype


def parse_header(file_path):
    
    column_names = []
    column_types = []
    col_def = False
    with open(file_path, 'r') as sql_dump:
        for line in sql_dump:
            if line.startswith('CREATE TABLE'):
                col_def = True
                continue
            if not col_def:
                continue
            if line.strip().startswith('KEY') or line.strip().startswith('UNIQUE'):
                break
            parts = line.strip().split()
            col_name = parts[0].strip('`')
            try:
                col_type = determine_python_type(parts[1])
            except ValueError:
                print(line)
                raise
            column_names.append(col_name)
            column_types.append(col_type)
    column_types = dict((n, t) for n, t in zip(column_names, column_types))
    return column_names, column_types

for dump in os.listdir(dump_path):
    if dump.endswith('sql') or dump.endswith('bed'):
        continue
    data_file = os.path.join(dump_path, dump)
    header_file = os.path.join(dump_path, dump.replace('txt.gz', 'sql'))
    column_names, column_types = parse_header(header_file)
    df = pd.read_csv(data_file, sep='\t', names=column_names, dtype=column_types)
    df.drop('bin', axis=1, inplace=True)
    if 'ix' in df:
        df.drop('ix', axis=1, inplace=True)
    if 'posBasesHit' in df:
        df.drop(['posBasesHit', 'testResult', 'verdict', 'chits', 'ccov', 'alignfile'],
                inplace=True, axis=1)
    
    output_file = data_file.replace('txt.gz', 'bed')
    with open(output_file, 'w') as bed:
        _ = bed.write('#')
        df.to_csv(bed, sep='\t', header=True, index=False)


short_reads
{'readset': 'NA12878_ptg_il2k-100pe_short', 'source_type': 'ena', 'bioproject': 'PRJEB3381', 'load_type': 'complete'}
